### **imports**

In [8]:
import pandas as pd
import numpy as np
from astropy.io import ascii
from astropy.table import QTable, Table, Column
from astropy import units as u
import string
import os
import upsilon

### **making a table of all of the stars in the txt file**

In [9]:
column1 = []
column2 = []
column3 = []
column4 = []
column5 = []

with open('rrlyr_vsx_clean2.txt', 'r') as file:
    for line in file:
        components = line.split(' ', 4)
    
        try:
            magnitude = float(components[3])  # Try to convert the 4th component to float
        except ValueError:
            continue

        if magnitude < 15.5:
            column1.append(components[0])
            column2.append(float(components[1]))
            column3.append(float(components[2]))
            column4.append(magnitude)
            column5.append(components[4].strip())

final_table = QTable([column1, column2, column3, column4, column5], names = ("Star Type", "RA", "Dec", "Magnitude", "Other Info"))

In [10]:
final_table

Star Type,RA,Dec,Magnitude,Other Info
str24,float64,float64,float64,str29
RRAB,0.01521,35.36286,12.43,GM And
RRAB,0.01669,18.40698,15.171,CSS_J000004.0+182425
RRAB,0.03171,34.67397,14.95,SERIV 27
RRAB,0.06821,-72.77875,15.1,BP Tuc
RRAB,0.099,36.3287,14.46,NSVS 6313844
RRAB,0.1338,22.99388,14.671,CSS_J000032.1+225937
RRAB,0.13423,-54.29512,15.26,ASASSN-V J000032.21-541742.4
RRAB,0.14829,26.66375,12.96,GV Peg
RRC,0.15377,-60.98439,15.18,ASASSN-V J000036.92-605903.9


### **importing the asassn client**

In [11]:
from pyasassn.client import SkyPatrolClient	
client = SkyPatrolClient()

Welcome to ASAS-SN Skypatrol!

Current Deployment Version: 0.6.17 (26 JAN 2024)
Please upgrade your client if not up to date.



### **downloading a csv file with the data from the star at a specified index**

In [12]:
def classification_model(path, ra, dec, magnitude1, vsx_id, asassn_id):
  # Load a classification model.
  # (takes time, do only once if processing multiple lcs)
  rf_model = upsilon.load_rf_model()


  ############# Read input lc data #############
  # Read the input lightcurve file
  f = open(path, 'r')
  data = np.loadtxt(f, dtype={'names': ('JD', 'mag', 'err'),'formats': ('float64', 'float64', 'float64')})
  f.close()

  # Split it into three arrays
  date=data['JD']
  mag=data['mag']
  err=data['err']

  # Sigma-clip the input lightcurve
  date, mag, err = upsilon.utils.sigma_clipping(date, mag, err,  threshold=3, iteration=1)

  ############# Extract lightcurve features #############
  e_features = upsilon.ExtractFeatures(date, mag, err)
  e_features.run()
  features = e_features.get_features()

  ############# Classify #############
  # Classify the light curve
  label, probability, flag = upsilon.predict(rf_model, features)

  # Print results

  output_file = open(path.replace('.txt', '_classification.txt'), 'w')
  output_file.write("\n\n######### Classification results #########\n")
  output_file.write("flag = " + str(flag) + " (0 - classification success, 1 - suspicious period)\n")
  output_file.write("class = " + str(label) + "\n")
  output_file.write("class_probability = " + str(probability) + "\n")
  output_file.write("position = " + str(ra) + " " + str(dec) + "\n")
  output_file.write("magnitude = " + str(magnitude1) + "\n")
  output_file.write("vsx_id = " + str(vsx_id) + "\n")
  output_file.write("asassn_id = " + str(asassn_id) + "\n")
  output_file.write("color_string =\n")
  output_file.write("visual_inspection_comments =\n")
  output_file.close()

  print("\n\n######### Classification results #########\n")
  print("flag = " + str(flag) + " (0 - classification success, 1 - suspicious period)")
  print("class = " + str(label) )
  print("class_probability = " + str(probability) )
  print("\n\n######### Lightcurve features #########\n")

  for feature in features:
    print( str(feature) + " = " + str(features[feature]) )
  #  print feature," = ",features[feature]

In [13]:
folder_path = "/Users/zoesurles/Desktop/variable_star_classification/downloaded_lightcurves/"

In [14]:
num_iterations = 1
empty_list = []
for i in range(2000, 2100):

    try:
        search = client.cone_search(ra_deg = final_table["RA"][i], dec_deg = final_table["Dec"][i], radius=0.0003, catalog='aavsovsx', download=True)
    except:
        print("\n ################################################## error in index", i, "\n")
        continue

    savecsv = search.save(save_dir = folder_path, file_format="csv")[1]
    empty_list.append(savecsv)

    new_file = pd.read_csv(savecsv, skiprows=1)

    g_filter = new_file[new_file['phot_filter'] == 'g']
    quality = g_filter[g_filter['quality'] == 'G']
    web_format = quality[["jd", "mag", "mag_err"]]
    web_format.to_csv(savecsv.replace(".csv", ".txt"), sep='\t', index=False, header = False)

    classification_model(savecsv.replace(".csv", ".txt"), final_table["RA"][i], final_table["Dec"][i], final_table["Magnitude"][i], final_table["Other Info"][i], new_file["asas_sn_id"][0])

Pulled 1 of 1


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_ab
class_probability = 1.0


######### Lightcurve features #########

amplitude = 0.4692631784844826
cusum = 0.26922765303645024
eta = 1.4168308688928095
hl_amp_ratio = 0.30077660993285404
kurtosis = -0.6372652628985813
n_points = 1303
period = 0.4891287556183197
period_SNR = 209.61482267375496
period_log10FAP = -173.98510187691585
period_uncertainty = 9.867435455404583e-05
phase_cusum = 0.30524012107375065
phase_eta = 0.20026961487494557
phi21 = -2.5605171997919163
phi31 = -1.6241529789995823
quartile31 = 0.6424168994284969
r21 = 0.4422829398340116
r31 = 0.34942804605990796
shapiro_w = 0.8922900557518005
skewness = -0.7563134528888922
slope_per10 = -0.023473744295457306
slope_per90 = 0.032572398581995676
stetson_k = 0.6562438820444345
weighted_mean = 15.386007171877882
weighted_std = 0.442134212025096
Pulled 1 of 1


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = NonVar
class_probability = 0.54


######### Lightcurve features #########

amplitude = 0.08950511362148775
cusum = 0.4942670808503735
eta = 2.2766033950622973
hl_amp_ratio = 1.1297371218747474
kurtosis = 0.01102806765246589
n_points = 928
period = 0.21899735062848052
period_SNR = 110.33639852675208
period_log10FAP = -54.737441061651964
period_uncertainty = 1.9995497378424987e-05
phase_cusum = 0.3661201680962735
phase_eta = 2.2331837524969216
phi21 = -0.6524611660436078
phi31 = -0.6701093895614938
quartile31 = 0.15835382518900687
r21 = 0.11738330499837164
r31 = 0.10673303311403952
shapiro_w = 0.988581657409668
skewness = 0.36713749639030413
slope_per10 = -0.014269462843339966
slope_per90 = 0.014645016783354768
stetson_k = 0.836249228672609
weighted_mean = 11.403715783766486
weighted_std = 0.09452355611106228
Pulled 1 of 1


...

/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_ab
class_probability = 0.98


######### Lightcurve features #########

amplitude = 0.40008257504668276
cusum = 0.24470717352594462
eta = 1.4950273201992348
hl_amp_ratio = 0.3492182321018168
kurtosis = -0.6995790698466422
n_points = 432
period = 0.4928329323823701
period_SNR = 109.48324850210342
period_log10FAP = -52.85288093303076
period_uncertainty = 0.00010066717905868328
phase_cusum = 0.40730420710507065
phase_eta = 0.49296812226566183
phi21 = 0.5759971220423411
phi31 = -1.5448888961098621
quartile31 = 0.5992612303197049
r21 = 0.37781313052638454
r31 = 0.19951657171935575
shapiro_w = 0.9263135194778442
skewness = -0.6310425690026302
slope_per10 = -0.043345704101175865
slope_per90 = 0.04614858606485232
stetson_k = 0.6657205471370685
weighted_mean = 15.99195591921529
weighted_std = 0.38469893768535357
Pulled 1 of 1


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_ab
class_probability = 0.99


######### Lightcurve features #########

amplitude = 0.26112539009211005
cusum = 0.14380351938692873
eta = 2.1329585438561067
hl_amp_ratio = 0.49598991377101176
kurtosis = -0.14347982554763572
n_points = 1716
period = 0.5758959653629674
period_SNR = 216.20333458738816
period_log10FAP = -151.69300087263224
period_uncertainty = 0.0001367815686446061
phase_cusum = 0.26065323191505735
phase_eta = 1.056757264028045
phi21 = -2.659952623652522
phi31 = -5.276553380858498
quartile31 = 0.4038282632150363
r21 = 0.3400895635605416
r31 = 0.15638110779211398
shapiro_w = 0.9618303775787354
skewness = -0.6425467598354572
slope_per10 = -0.011883192385614177
slope_per90 = 0.009577883899140537
stetson_k = 0.7044193786138754
weighted_mean = 15.88081623976747
weighted_std = 0.28760551677767426


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
